# Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import json

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings(action='ignore')


from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.ensemble import forest
from sklearn.tree import export_graphviz

from collections import defaultdict

from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Dataset
from scipy.linalg import svd
from surprise import Reader
from surprise.model_selection import cross_validate

# 1. Import the necessary libraries and read the provided CSVs as a data frame and perform the below steps. [15 Marks]

## A. Merge all the provided CSVs into one dataFrame. [2 Marks]

In [2]:
df1 = pd.read_csv("phone_user_review_file_1.csv",encoding='ISO-8859-1')
df2 = pd.read_csv("phone_user_review_file_2.csv",encoding='ISO-8859-1')
df3 = pd.read_csv("phone_user_review_file_3.csv",encoding='ISO-8859-1')
df4 = pd.read_csv("phone_user_review_file_4.csv",encoding='ISO-8859-1')
df5 = pd.read_csv("phone_user_review_file_5.csv",encoding='ISO-8859-1')
df6 = pd.read_csv("phone_user_review_file_6.csv",encoding='ISO-8859-1')

#### Check to see if all the DataFrames have the same columns

In [3]:
list1 = [df1,df2,df3,df4,df5,df6]
a = []
for i in list1:
    for j in i.columns:
        a.append(j)
        
if len(set(a)) == 11:
    print("All Dataframes have same columns")
else:
    print("There are different columns in Dataframes")

All Dataframes have same columns


In [4]:
a = []
for i in list1:
    a.append(len(i))

print("Total number of rows:", sum(a))

Total number of rows: 1415133


In [5]:
df = pd.concat(list1)
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [6]:
df.shape

(1415133, 11)

## B. Explore, understand the Data and share at least 2 observations. [2 Marks]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1415133 entries, 0 to 163836
Data columns (total 11 columns):
phone_url    1415133 non-null object
date         1415133 non-null object
lang         1415133 non-null object
country      1415133 non-null object
source       1415133 non-null object
domain       1415133 non-null object
score        1351644 non-null float64
score_max    1351644 non-null float64
extract      1395772 non-null object
author       1351931 non-null object
product      1415132 non-null object
dtypes: float64(2), object(9)
memory usage: 129.6+ MB


In [8]:
df['score_max'].value_counts()

10.0    1351644
Name: score_max, dtype: int64

In [9]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)

In [10]:
print("Range:",df['date'].min(), "to", df['date'].max())

Range: 1970-01-01 00:00:00 to 2017-12-05 00:00:00


In [11]:
df['lang'].value_counts()[:5]

en    554746
ru    207443
de    176600
it    116120
es     99739
Name: lang, dtype: int64

In [12]:
df['country'].value_counts()[:5]

us    318435
de    176525
ru    174945
in    127876
it    116120
Name: country, dtype: int64

#### Observations

* Score Max is the same throughout, so we can remove it.
* Source and Domain represent the same thing, so one of them can be dropped.
* The data is available from 1970 to 2017.
* The columns phone_url and Product also represnt the same thing.
* The most number of reviews are from US and in English.

## C. Round off scores to the nearest integers. [3 Marks]

In [13]:
df[['score','score_max']].head()

,score,score_max
0,10.0,10.0
1,10.0,10.0
2,6.0,10.0
3,9.2,10.0
4,4.0,10.0


In [14]:
df['score'].value_counts()

10.0    656239
8.0     296018
2.0     128485
6.0     116616
4.0      72462
         ...  
0.4          2
4.9          1
1.5          1
8.9          1
6.1          1
Name: score, Length: 86, dtype: int64

In [15]:
df[['score','score_max']] = round(df[['score','score_max']])

In [16]:
df[['score','score_max']].head()

,score,score_max
0,10.0,10.0
1,10.0,10.0
2,6.0,10.0
3,9.0,10.0
4,4.0,10.0


In [17]:
df['score'].value_counts()

10.0    665748
8.0     305550
2.0     128762
6.0     118349
4.0      73333
9.0      40602
7.0       8771
5.0       4730
1.0       3244
3.0       2513
0.0         42
Name: score, dtype: int64

## D. Check for missing values. Impute the missing values, if any. [2 Marks]

#### No. of missing values column wise:

In [18]:
df.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [19]:
df[['score','score_max']].median()

score         9.0
score_max    10.0
dtype: float64

In [20]:
df['author'].mode()[0]

'Amazon Customer'

#### Applying Mode for object based columns and median for float based columns:

In [21]:
for i in df.columns:
    if df[i].dtype=='float64':
        df[i] = df[i].fillna(df[i].median())
    else:
        df[i] = df[i].fillna(df[i].mode()[0])

#### Checking if any null values still left

In [22]:
df.isna().sum()

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64

## E. Check for duplicate values and remove them, if any. [2 Marks]

In [23]:
df.duplicated().sum()

9951

#### Dropping all the duplicate values (keeping only the first copy)

In [24]:
df.drop_duplicates(inplace=True)

In [25]:
df.duplicated().sum()

0

In [26]:
df.shape

(1405182, 11)

## F. Keep only 1 Million data samples. Use random state=612. [2 Marks]

In [27]:
df = df.sample(n=1000000, random_state=612)

In [28]:
df.shape

(1000000, 11)

## G. Drop irrelevant features. Keep features like Author, Product, and Score. [2 Marks]

In [29]:
df.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
80546,/cellphones/sony-xperia-go/,2013-11-10,fr,fr,SFR,avis.sfr.fr,9.0,10.0,"TRES BON INVESTISSEMENT, POUR UN PREMIER TACTILE",VALOU29,SONY Xperia Go Noir Reconditionne
175438,/cellphones/huawei-p8-lite/,2016-09-29,nl,nl,KIESKEURIG,kieskeurig.nl,1.0,10.0,"Wifi ontvangst niet goed, kijk maar op interne...",De vries,Huawei P8lite zwart / 16 GB
209133,/cellphones/lg-leon-h320/,2015-05-21,it,it,Ciao,ciao.it,6.0,10.0,LG Leon Ã¨ il nuovo ed innovativo smartphone A...,S_marts,LG Leon H320
57527,/cellphones/oneplus-two/,2015-12-23,en,us,Amazon,amazon.com,8.0,10.0,"Quiet nice phone: feels good in hands, top spe...",LSNPathfinder,OnePlus 2 Factory Unlocked A2005 4G Smartphone...
145421,/cellphones/lg-e900-optimus-7/,2011-10-13,de,de,Amazon,amazon.de,10.0,10.0,"...also ein Eifon haben ja mittlerweile viele,...",Ralf Menzel-Queck,LG Electronics E900 Optimus 7 Smartphone (Voda...


In [30]:
df = df[['author','product','score']]
df.head()

,author,product,score
80546,VALOU29,SONY Xperia Go Noir Reconditionne,9.0
175438,De vries,Huawei P8lite zwart / 16 GB,1.0
209133,S_marts,LG Leon H320,6.0
57527,LSNPathfinder,OnePlus 2 Factory Unlocked A2005 4G Smartphone...,8.0
145421,Ralf Menzel-Queck,LG Electronics E900 Optimus 7 Smartphone (Voda...,10.0


In [31]:
df.shape

(1000000, 3)

# 2. Answer the following questions. [10 Marks]

## A. Identify the most rated products. [3 Marks]

In [32]:
print("The 5 most rated products are:","\n",df['product'].value_counts()[:5])

The 5 most rated products are: 
 Lenovo Vibe K4 Note (White,16GB)     3724
Lenovo Vibe K4 Note (Black, 16GB)    3084
OnePlus 3 (Graphite, 64 GB)          2893
OnePlus 3 (Soft Gold, 64 GB)         2527
Huawei P8lite zwart / 16 GB          1902
Name: product, dtype: int64


### B. Identify the users with most number of reviews. [3 Marks]

In [33]:
print("The 5 users with the most number of reviews are:","\n",df['author'].value_counts()[:5])

The 5 users with the most number of reviews are: 
 Amazon Customer    98565
Cliente Amazon     13657
e-bit               6014
Client d'Amazon     5509
Amazon Kunde        3280
Name: author, dtype: int64


### C. Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final dataset. [4 Marks]

In [34]:
a = df['product'].value_counts() > 50
b = df['author'].value_counts() > 50
x = a[a].index
y = b[b].index

print("Number of Unique products with more than 50 ratings:",x.shape[0])
print("Number of users with more than 50 ratings:",y.shape[0])

Number of Unique products with more than 50 ratings: 4253
Number of users with more than 50 ratings: 644


In [35]:
df = df[(df['product'].isin(x)) & (df['author'].isin(y))]

In [36]:
print("The shape of the final dataset is:", df.shape)

The shape of the final dataset is: (129577, 3)


## 3. Build a popularity based model and recommend top 5 mobile phones. [5 Marks]

In [37]:
df.head()

,author,product,score
32671,sozer,Nokia N72 Cep Telefonu,9.0
67017,Cliente Amazon,"Huawei P9 Lite Smartphone, LTE, Display 5.2'' ...",10.0
365805,Amazon Customer,"OnePlus X (Onyx, 16GB)",10.0
160601,M,Sim Free Samsung Galaxy A3 2016 Mobile Phone -...,10.0
3172,kuzeykizi,Nokia N95 8 GB Cep Telefonu,9.0


#### Top 5 Mobile phones based on average ratings

In [38]:
df.groupby('product')['score'].mean().sort_values(ascending=False).head()

product
3100 / 3120                                                     10.0
BlackBerry OEM Z10 Black Leather Swivel Holster                 10.0
Sony Ericsson V800                                              10.0
Sony Ericsson T230                                              10.0
MOTOROLA Moto G XT 1032 8 GB Black (Charger sold seperately)    10.0
Name: score, dtype: float64

#### Top 5 Mobile phones based on number of ratings provided

In [39]:
df.groupby('product')['score'].count().sort_values(ascending=False).head()

product
Lenovo Vibe K4 Note (White,16GB)       2197
Lenovo Vibe K4 Note (Black, 16GB)      1785
OnePlus 3 (Graphite, 64 GB)            1308
OnePlus 3 (Soft Gold, 64 GB)           1236
Lenovo Vibe K5 (Gold, VoLTE update)    1113
Name: score, dtype: int64

#### Top 5 Mobile phones based on mean Ratings and number of ratings provided (>100 ratings)

In [40]:
a = df.groupby('product')['score'].mean().sort_values(ascending=False).head()
b = df.groupby('product')['score'].count().sort_values(ascending=False).head()>100

x = a[a].index
y = b[b].index

In [41]:
df[(df['score'].isin(x)) & (df['product'].isin(y))].head()

,author,product,score
51068,Amazon Customer,"Lenovo Vibe K4 Note (White,16GB)",10.0
18425,Amazon Customer,"Lenovo Vibe K5 (Gold, VoLTE update)",10.0
55627,Amazon Customer,"Lenovo Vibe K4 Note (White,16GB)",10.0
72806,Amazon Customer,"OnePlus 3 (Soft Gold, 64 GB)",10.0
55699,Amazon Customer,"Lenovo Vibe K4 Note (White,16GB)",10.0


## 4. Build a collaborative filtering model using SVD. You can use SVD from surprise or build it from scratch(Note: Incase you’re building it from scratch you can limit your data points to 5000 samples if you face memory issues). Build a collaborative filtering model using kNNWithMeans from surprise. You can try both user-based and item-based model. [10 Marks]

#### SVD

In [42]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['author', 'product', 'score']], reader)

In [43]:
trainset, testset = train_test_split(data, test_size=0.15)

In [44]:
algo = SVD()
algo.fit(trainset)

In [45]:
predictions = algo.test(testset)

#### KNNWithMeans
#### User-Based

In [46]:
algo1 = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo1.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [47]:
predictions1 = algo1.test(testset)

#### Item-Based

In [48]:
algo2 = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo2.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [49]:
predictions2 = algo2.test(testset)

## 5. Evaluate the collaborative model. Print RMSE value. [2 Marks]

In [50]:
print("SVD-based Model : Test Set")
accuracy.rmse(predictions, verbose=True)

SVD-based Model : Test Set
RMSE: 2.6166


2.616574170045788

In [51]:
print("User-based Model : Test Set")
accuracy.rmse(predictions1, verbose=True)

User-based Model : Test Set
RMSE: 2.6691


2.6691070135218604

In [52]:
print("Item-based Model : Test Set")
accuracy.rmse(predictions2, verbose=True)

Item-based Model : Test Set
RMSE: 2.6097


2.6096704977583984

## 6. Predict score (average rating) for test users. [2 Marks]

In [53]:
ratings = []
for uid, iid, r_ui, est, _ in predictions:
    ratings.append(est)

print("The average ratings for test users for SVD-Based Model:",np.mean(ratings))

The average ratings for test users for SVD-Based Model: 7.851738227694242


In [54]:
ratings = []
for uid, iid, r_ui, est, _ in predictions1:
    ratings.append(est)

print("The average ratings for test users for User-Based Model:",np.mean(ratings))

The average ratings for test users for User-Based Model: 7.894056813868899


In [55]:
ratings = []
for uid, iid, r_ui, est, _ in predictions2:
    ratings.append(est)

print("The average ratings for test users for Item-Based Model:",np.mean(ratings))

The average ratings for test users for Item-Based Model: 7.8847194424471665


## 7. Report your findings and inferences. [2 Marks]

* Looking at the RMSE values all the three models SVD, User-based and Item-Based Collaborative filtering recommendation systems are giving nearly same results.
* The average ratings are also almost the same.
* The results of the three above mentioned models is very different to that of the Popularity based model, as the latter was only giving results based on means.
* The recommendations could be further improved using cross-validation technique which we will explore further.

## 8. Try and recommend top 5 products for test users. [5 Marks]

In [56]:
def get_top_n(predictions, n):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [57]:
top_n = get_top_n(predictions, 5)
print("Predicting top 5 products for SVD-based test users:")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print("==============================================")

Predicting top 5 products for SVD-based test users:
Ð¡ÐµÑÐ³ÐµÐ¹ ['Samsung Galaxy Note5', 'Sony Xperia Z1 Compact (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Sony Xperia Z1 Compact (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Nokia Lumia 920 White', 'Sony Xperia C (Ñ\x84Ð¸Ð¾Ð»ÐµÑ\x82Ð¾Ð²Ñ\x8bÐ¹)']
ÐÐ½Ð´ÑÐµÐ¹ ['Samsung Galaxy Note5', 'Samsung Galaxy Note5', 'Samsung Galaxy A3', 'Samsung S7270 Galaxy Ace 3 (ÐºÑ\x80Ð°Ñ\x81Ð½Ñ\x8bÐ¹)', 'Sony Xperia Z1 Compact (Ð±ÐµÐ»Ñ\x8bÐ¹)']
James ['Microsoft Nokia Lumia 630 Single-SIM Smartphone (4,5 Zoll (11,4 cm) Touch-Display, 8 GB Speicher, Windows 8.1) gelb', 'Sim Free Motorola Moto G 4th Generation Mobile Phone - Black', 'ZTE Axon 7 64GB Smartphone (Unlocked, Quartz Gray)', 'LG Voyager', 'Samsung Galaxy S5 SM-G900A GSM Unlocked Cellphone, 16GB, Black']
Franco ['Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]', 'Samsung Galaxy S II Nero GT-I9100', 'Huawei Ascend G525 Smartphon

Tim ['LG G4 US991 32GB Smartphone (Unlocked, Black Leather)', 'Lenovo Motorola Moto G 2. Generation Dual-Sim Smartphone (5 Zoll (12,7 cm) Touch-Display, 8 GB Speicher, Android 5.02) schwarz', 'Nokia Lumia 800 Cyan Blue', 'Sony Xperia Z5 zwart / 32 GB', 'Sony Ericsson Xperia mini pro Smartphone (7,6 cm (3 Zoll) Display, QWERTZ-Tastatur, Touchscreen, 5 MP Kamera, Android 2.3 OS) schwarz']
ÐÐ°ÑÐ¸Ð½Ð° ['Sony Xperia L (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Sony Xperia Z1 Compact (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)']
jose ['Lenovo Moto G4 Play - Smartphone de 5 (4G, RAM de 2 GB, memoria interna de 16 GB, cÃ¡mara de 8 MP, Android 6), blanco', 'Nokia Lumia 1520', 'Sony Xperia T - Smartphone libre Android (pantalla de 4,55", cÃ¡mara 13 Mp, 16 GB, Procesador de doble nÃºcleo Qualcomm , 1 GB de RAM), negro', 'Motorola Nexus 

Luis ['LG G2 D800 GSM 4G LTE Unlocked Smartphone with 13MP Camera, 32GB, Black', 'Desconocido Xiaomi MI 3 - Smartphone libre Android (pantalla 5", cÃ¡mara 13 Mp, 16 GB, Quad-Core 2.3 GHz, 2 GB RAM), blanco', 'Samsung Galaxy S Duos (S7562) - Smartphone libre Android (pantalla 4", c??mara 5 Mp, 4 GB, 1 GHz), negro [importado]', 'BLU LIFE 8 Unlocked (White)', 'Sony Xperia M2 - Smartphone libre Android (Pantalla 4.8", cÃ¡mara 8 Mp, 8 GB, Quad-Core 1.2 GHz, 1024 MB RAM), pÃºrpura (importado)']
Laurent ['BlackBerry Curve 8900', "Samsung B500BE Batterie d'origine pour Samsung Galaxy S4 Mini Noir"]
Angel ['Smartphone Samsung Galaxy S III Neo Duos Branco com Tela 4.8", Dual Chip, CÃ¢mera 8MP, Android 4.3, 3G e Processador Quad-Core de 1.4 Ghz', 'Sony Xperia L - Smartphone libre Android (pantalla 4.3", cÃ¡mara 8 Mp, 8 GB, Dual-Core 1 GHz, 1 GB RAM), blanco (importado)', 'Samsung Galaxy S5 Mini G800H Unlocked Cellphone, International Version, 16GB, Black', 'SONY XPERIA Z3 D6603 LTE 16GB 5.2" GSM 

In [58]:
top_n = get_top_n(predictions1, n=5)
print("Predicting top 5 products for User-based test users:")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print("==============================================")

Predicting top 5 products for User-based test users:
Ð¡ÐµÑÐ³ÐµÐ¹ ['Nokia Lumia 920 White', 'Samsung Galaxy Note5', 'Samsung S5570 Galaxy Mini', 'Nokia Lumia 920', 'Samsung Galaxy S7']
ÐÐ½Ð´ÑÐµÐ¹ ['Nokia Lumia 920', 'Samsung Galaxy Note5', 'Samsung Galaxy Note5', 'Samsung Galaxy A3', 'Sony Xperia Z1 Compact (Ð±ÐµÐ»Ñ\x8bÐ¹)']
James ['Samsung Galaxy Note 3 N900A Unlocked Cellphone, 32GB, White', 'Nokia Lumia 635 (AT&T Go Phone) No Annual Contract', 'Samsung Galaxy S5 SM-G900A GSM Unlocked Cellphone, 16GB, Black', 'Sim Free Apple iPhone SE 64GB Mobile Phone - Space Grey', 'Samsung Galaxy Exhibit 4G (T-Mobile), t679']
Franco ['Samsung Galaxy S6 Edge G925F 32GB Unlocked GSM LTE Octa-Core Phone - White', 'Samsung Galaxy S4 i9500 Factory Unlocked Cellphone, International Version, 16GB, Black', 'Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]', 'Huawei P8 lite Smartphone, Display 5.0" IP

ÐÑÐ¸Ð½Ð° ['Samsung Galaxy A5 (2016)', 'Samsung Galaxy A5 (2016)', 'Apple iPhone SE 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Sony Xperia L (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Sony Xperia L (Ð±ÐµÐ»Ñ\x8bÐ¹)']
ÐÑÑÑÐ¼ ['Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Sony Xperia ZR (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Sony Xperia C (Ñ\x84Ð¸Ð¾Ð»ÐµÑ\x82Ð¾Ð²Ñ\x8bÐ¹)', 'Sony Xperia ZR (Ð±ÐµÐ»Ñ\x8bÐ¹)']
ÐÐ³Ð¾ÑÑ ['HTC Desire', 'Sony Xperia M (Ñ\x84Ð¸Ð¾Ð»ÐµÑ\x82Ð¾Ð²Ñ\x8bÐ¹)', 'LG Optimus L7 P705 Black', 'Sony Xperia Z (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Samsung Galaxy S4 mini GT-I9190 (Ñ\x81Ð¸Ð½Ð¸Ð¹)']
??????????? ['LG Optimus L5 E612 White', 'Sony Xperia V (?\x80????????????)', 'Sony Xperia V (?????\x80??????)', 'Sony Xperia V (?????\x80??????)', 'Sony Xperia V (?????\x80??????)']
ÐÐ»Ð¸Ð½Ð° ['Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Prestigio MultiPhone 4040 DUO', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81

Sarah ['Sony Ericsson Xperia pro Smartphone (9.4 cm (3.7 Zoll) Tochscreen, 8.1 Megapixel Kamera, Android 2.3) silber', 'Doogee X5 MAX 3G Smartphone MTK6580 Quad Core Android 6,0 1G + 8G 5.0" IPS HD 1280 * 720 Pixels d\'Ã©cran 8MP + 8MP Double CamÃ©ras Empreintes Digitalesle Geste Intelligent', 'Tech Armor Samsung Galaxy S3 S III Premium Anti-Glare & Anti-Fingerprint (Matte) Screen Protectors with Lifetime Replacement Warranty [3-PACK] - Retail Packaging', 'Samsung B2100 Solid Extreme Sim Free Mobile Phone', 'HTC Desire X Wit']
ÐÐ¸ÐºÑÐ¾ÑÐ¸Ñ ['Samsung Galaxy A3 (2016)', 'Samsung I9082 Galaxy Grand Duos (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Samsung Galaxy S4 mini GT-I9190 (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Samsung Galaxy S5 SM-G900F 16GB (Ð·Ð¾Ð»Ð¾Ñ\x82Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Samsung N7100 Galaxy Note II 16GB (Ð±ÐµÐ»Ñ\x8bÐ¹)']
sara ['Lenovo Motorola Moto G Smartphone, 4,5 pollici display HD, processore Qualcomm, memoria 16GB, MicroSIM, Android 4.3 OS, fotocamera da 5 MP, Nero [Germania]', 'Lenovo Motorola Moto G Smartph

sumeyyehavvaay ['Nokia 6288 Cep Telefonu']
Vanessa ['Motorola Moto X - 16GB, Unlocked Phone - US Warranty - Black', 'Kyocera Clip S2100 Prepaid Phone (payLo by Virgin Mobile)', 'Samsung Galaxy S3 mini I8190 Smartphone (10,2 cm (4 Zoll) AMOLED Display, Dual-Core, 1GHz, 1GB RAM, 5 Megapixel Kamera, Android 4.1) marble-white', 'Sony Xperia Z1 - Smartphone libre Android (pantalla 5", cÃ¡mara 20.7 Mp, 16 GB, 2.2 GHz, 2 GB RAM), blanco (importado)', 'Nokia N97']
Josh ['Motorola Moto Z Play - Black - 32GB (U.S. Warranty)', 'Tech Armor Samsung Galaxy S3 S III Premium High Definition (HD) Clear Screen Protectors with Lifetime Replacement Warranty [3-PACK] - Retail Packaging', 'LG Optimus L9 P769', 'Honor Huawei Honor 6X Dual Camera Unlocked Smartphone, 32GB Gray (US Warranty)']
Rafael ['Elephone P6000 Pro Smartphone 4G, 5.0 Pulgadas 64bit Android 5.1 MTK6753 1.5GHz Octa Core 3GB 16GB Doble CÃ¡maras (Blanco)', 'Apple iPhone 6 64GB', "Samsung Galaxy A3 (2016) - Smartphone libre Android (4.7'', 13

In [59]:
top_n = get_top_n(predictions2, n=5)
print("Predicting top 5 products for Item-based test users:")
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])
    print("==============================================")

Predicting top 5 products for Item-based test users:
Ð¡ÐµÑÐ³ÐµÐ¹ ['Samsung Galaxy Note5', 'Nokia Lumia 920 White', 'Samsung Galaxy S7', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)']
ÐÐ½Ð´ÑÐµÐ¹ ['Samsung Galaxy Note5', 'Samsung Galaxy Note5', 'Samsung Galaxy S8+', 'Samsung S7270 Galaxy Ace 3 (ÐºÑ\x80Ð°Ñ\x81Ð½Ñ\x8bÐ¹)', 'Sony Xperia Z (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)']
James ['ZTE Axon 7 64GB Smartphone (Unlocked, Quartz Gray)', 'BlackBerry Z30 Factory Unlocked Black - 16GB', 'Sim Free Apple iPhone SE 64GB Mobile Phone - Space Grey', 'Sim Free Motorola Moto G 4th Generation Mobile Phone - Black', 'LG Voyager']
Franco ['Lenovo Motorola Moto G LTE Smartphone, Display 4.5", Fotocamera 5 MP, Memoria 8 GB, Quad-Core 1.2 GHz, 1 GB RAM, Micro SD, Android 4.4.3, Bianco [Italia]', 'Huawei Ascend G525 Smartphone, 4 GB, Dual SIM, Bianco', 'Samsung Galaxy S II Nero GT-I9100', 'Samsung Galaxy S4 i9500 Factory Unlocked C

Benjamin ['Pantech Pocket P9060 Unlocked GSM Phone with Android 2.3 OS, Touchscreen, 5MP Camera, Video, GPS, Wi-Fi, SNS integration, MP3/MP4 Player and microSD Slot - Gray', 'APPLE iPhone 7 - Silver, 128 GB', 'LG G Vista 4G Unlocked 4G LTE Android 8MP Camera Smartphone w/ Quad-Core CPU - Black', 'Microsoft Nokia C2-01 Handy (Ohne Branding, 5,1 cm (2 Zoll), 3,2 Megapixel Kamera) schwarz', 'Honor 7 Smartphone dÃ©bloquÃ© 4G (Ecran: 5,2 pouces - 16 Go - Double Nano SIM - Android 5.0 Lollipop) Gris/Noir']
ÐÐ½Ð°ÑÐ¾Ð»Ð¸Ð¹ ['Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Sony Xperia Z1 Compact (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Samsung Galaxy S4 GT-I9500 16GB (Ð±ÐµÐ»Ñ\x8bÐ¹)']
ÐÑÑÐµÐ¼ ['Samsung Galaxy Note5', 'Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)', 'Samsung Galaxy S7', 'HTC Evo 3D', 'Xiaomi Mi4i 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹)']
Ali 

Marcel ['Huawei P8lite zwart / 16 GB', 'Huawei P8 grijs / 16 GB', 'HTC Wildfire', 'Huawei P8 Lite Black', 'Doro PhoneEasy 610gsm Mobiltelefon (UKW, Bluetooth, GSM) schwarz']
Deepak ['Samsung Galaxy J3 (8GB)', 'OnePlus 3 (Soft Gold, 64 GB)', 'OnePlus 3T (Soft Gold, 6GB RAM + 64GB memory)']
monica ['OnePlus X (Onyx, 16GB)']
Sabine ['Samsung E1100 Handy black']
Ð¡Ð°ÑÐ° ['Apple iPhone 5s 16GB (Ñ\x81ÐµÑ\x80Ñ\x8bÐ¹ ÐºÐ¾Ñ\x81Ð¼Ð¾Ñ\x81)', 'Samsung Galaxy J5 (2016)', 'LG K7 (Ñ\x87ÐµÑ\x80Ð½Ñ\x8bÐ¹)', 'Samsung I9082 Galaxy Grand Duos (Ð±ÐµÐ»Ñ\x8bÐ¹)', 'Samsung Galaxy S4 mini Duos GT-I9192 (Ð±ÐµÐ»Ñ\x8bÐ¹)']
sezenm ['Samsung P520 Armani Cep Telefonu', 'Apple iphone 3G 16GB Cep Telefonu']
eduardo ['Huawei P8 Lite-AM08 - Pack Smartphone Huawei P8 Lite negro con Altavoz portÃ¡til AM08 Gold (2 GB de RAM, 16 GB de memoria interna, cÃ¡mara 13 Mpxs/8Mpx), color negro', 'BQ Aquaris E5 HD - Smartphone libre Android (pantalla 5 pulgadas, cÃ¡mara 13 Mp, 16 GB, Quad-Core 1.3 GHz, 1 GB RAM), blanco y negro', '

## 9. Try other techniques (Example: cross validation) to get better results. [3 Marks]

In [60]:
svd = SVD(n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    2.5839  2.5969  2.6045  2.5951  0.0085  
MAE (testset)     2.0489  2.0612  2.0307  2.0469  0.0125  
Fit time          4.17    4.22    4.72    4.37    0.25    
Test time         0.52    0.67    1.04    0.74    0.22    


{'test_rmse': array([2.58390263, 2.59691671, 2.60447618]),
 'test_mae': array([2.04886355, 2.0611867 , 2.03072746]),
 'fit_time': (4.165911674499512, 4.2224767208099365, 4.717500686645508),
 'test_time': (0.5175387859344482, 0.6720812320709229, 1.0448434352874756)}

In [61]:
svd.fit(trainset)

In [62]:
predictions3 = svd.test(testset)
print(accuracy.rmse(predictions3))

RMSE: 2.5665
2.566479168881502


## 10. In what business scenario you should use popularity based Recommendation Systems ? [2 Marks]

**As the name suggests Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now. For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up.**

**Business Scenario:** 
Suppose a new user signs up on Netflix, now the platform does not has any data on the user as he is new and has not rated any movie nor mentioned any details about his likes or dislikes. In this case, Netflix does not has data to apply on any advanced algorithms to make suggestions to this new user. Hence, popularity base recommendation systems can be helpful here. It will browse through all the movies and based on popularity, trands or current ratings recommend the movies to that user. It is not a very custom based or advanced approach but surely increases the probability a lot higher for the user to watch a recommended movie than earlier.

## 11. In what business scenario you should use CF based Recommendation Systems ? [2 Marks]

**Collaborative filter based recommendation systems works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user. It looks at the items they like and combines them to create a ranked list of suggestions.**

**Business Scenario:**
Lets take the case of Amazon, suoopose a user buys a playstation game on amazon. The challenge here would be to recommend the user with a product that is very close to his likings based on his Pusrchase. Here collaborative filtering based recommendation systems can be helpful. Since an e-commerce platform like Amazon has millions of users surfing through the platform, this technique is of great use to the company and its users. Amazon offers a user-based approach and an item-based approach for suggested products and similar items. The platform's item-based collaborative filtering has proved to be a useful system that has triggered the profit-making capacity of the business. 

## 12. What other possible methods can you think of which can further improve the recommendation for different users ? [2 Marks]

**Some of the methods that come in mind which can further improve the recommendation to different users:**

* In the case of user-bassed collaborative filter algorithms, a key challenge we face in a business setting is tackling data sparsity and scalability. In a user-based collaborative filtering models the problem largely arises in the search for neighbours, which are other users who have historically shown similar preferences to a given user, among large user populations. In this case switching to an item-based collabortive filtering algorithm can be helpful as it transposes the recommendation problem from user space to item space.

* Use of Hybrid recommendation algorithms can also be useful as it helps in diversifying from a cold start or a grey area problem to the general population.

* Again availablity of data is the biggest factor for most advanced recommendation systems algorithms, so more the data the better.

* Decision, hypothesizing, experimentation and measurement are also considered ways of improving the performance of the recommendation systems. This includes the decisions on how we want it to improve, as a pre-processing step hypothsizing on what changes we want to make in our system to help improve the metrics. Implementing the above hypothesis and experimenting on it and at last to measure whether the metrics are improving.